In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import pickle
from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance
from tqdm import tqdm
from pathlib import Path
import numpy as np
import pandas as pd
import math

In [3]:
cat_slides = [
                "BAL_Cat_Turnbull blue_13.svs", 
                "BAL_Cat_Turnbull blue_6.svs", 
                "BAL_Cat_Turnbull blue_1.svs", 
                "BAL_Cat_Turnbull blue_12.svs", 
                "BAL_Cat_Turnbull blue_14.svs", 
                "BAL_Cat_Turnbull blue_10.svs", 
                "BAL_Cat_Turnbull blue_2.svs",]

In [4]:
fold_1 = pickle.load(open(str("inference_results_Equine-Fold-1.p"),'rb'))
fold_2 = pickle.load(open(str("inference_results_Equine-Fold-2.p"),'rb'))
fold_3 = pickle.load(open(str("inference_results_Equine-Fold-3.p"),'rb'))

In [5]:
folds = {}
nms_thresh = 40
thresh = 0.35


for file_name in tqdm(cat_slides):
    
    boxes1 = np.array(fold_1[file_name])
    boxes2 = np.array(fold_2[file_name])
    boxes3 = np.array(fold_3[file_name])
    
    boxes = np.concatenate([boxes1, boxes2, boxes3])
    init_shape = boxes.shape
    
    scales = ((boxes[:, 2] - boxes[:, 0]) / (boxes[:, 3] - boxes[:, 1]))
    boxes = boxes[(scales < 1.1) & (scales > 0.9)]
    
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]
    final_shape = annos.shape    
    
    #print("Input: {}  Output: {}".format(init_shape, final_shape))
    
    folds[file_name] = annos

100%|██████████| 7/7 [00:30<00:00,  3.30s/it]


In [6]:
pickle.dump(folds, open(str("InferenceAnnotationsCatV2.pickle"), "wb"))

In [6]:
folds = pickle.load( open( "InferenceAnnotationsCatV2.pickle", "rb" ) )

In [7]:
annotations_path = Path("../../Statistics/EIPH_Cat_SDATA_Screening_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)

annotations.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
0,1049539,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 9903.0, 'x2': 10029.0, 'y1': 5508.0, 'y...",875aab61-3f1f-429a-a053-f87aba6c99da,1,False,1,SDATA,Screened
1,1049548,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 12243.0, 'x2': 12375.0, 'y1': 14740.0, ...",d718f087-7a46-4e22-8610-ba5f82d04818,1,False,1,SDATA,Screened
2,1049551,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 20253.0, 'x2': 20366.0, 'y1': 4149.0, '...",7e9cc21f-4a51-4d12-99dd-daffa3e6f368,1,False,1,SDATA,Screened
3,1049568,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 2906.0, 'x2': 3030.0, 'y1': 16767.0, 'y...",0a546034-021b-4739-82bf-b09c7a5226d5,1,False,1,SDATA,Screened
4,1049624,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 11203.0, 'x2': 11328.0, 'y1': 7867.0, '...",52536c1e-32f3-4a15-9c08-0350c0ad1b2b,1,False,1,SDATA,Screened


## find optionmal thresholds

In [8]:
def get_grade(gt, prediction):
    
    gt_mean = float(gt["grade"].mean())
    pred_mean = float(prediction[:, 4].mean())
    diff = abs(gt_mean-pred_mean)
    
    
    return gt_mean, pred_mean, diff 

In [12]:
nms_threshs = [40] 
threshs = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]

results = []
for file in tqdm(folds):
    
    dfImage = annotations[annotations["image_name"] == file] 

    for nms_thresh in nms_threshs:
        for thresh in threshs:
            
            boxes = np.array(folds[file])
            annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
            annos = annos[annos[:, 5] > thresh]
            
            gt_mean, pred_mean, diff = get_grade(dfImage, annos)
            results.append([file, nms_thresh, thresh, gt_mean, pred_mean, diff, len(dfImage), len(annos), abs(len(dfImage) - len(annos))])
            
results = pd.DataFrame(results, columns=["image_name", "nms_thresh", "thresh", "gt", "pred", "diff", "num_gt", "num_pred", "diff_num"])
results

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:53<00:00,  7.68s/it]


,image_name,nms_thresh,thresh,gt,pred,diff,num_gt,num_pred,diff_num
0,BAL_Cat_Turnbull blue_13.svs,40,0.35,0.148676,0.132549,0.016127,5475,18695,13220
1,BAL_Cat_Turnbull blue_13.svs,40,0.40,0.148676,0.135544,0.013132,5475,17854,12379
2,BAL_Cat_Turnbull blue_13.svs,40,0.45,0.148676,0.137263,0.011413,5475,17033,11558
3,BAL_Cat_Turnbull blue_13.svs,40,0.50,0.148676,0.139403,0.009273,5475,16169,10694
4,BAL_Cat_Turnbull blue_13.svs,40,0.55,0.148676,0.140742,0.007934,5475,15198,9723
...,...,...,...,...,...,...,...,...,...
79,BAL_Cat_Turnbull blue_2.svs,40,0.70,0.917399,0.808446,0.108953,569,6938,6369
80,BAL_Cat_Turnbull blue_2.svs,40,0.75,0.917399,0.836421,0.080978,569,5074,4505
81,BAL_Cat_Turnbull blue_2.svs,40,0.80,0.917399,0.878533,0.038866,569,2972,2403
82,BAL_Cat_Turnbull blue_2.svs,40,0.85,0.917399,0.901676,0.015723,569,895,326


In [13]:
nms_threshs = [40] 
threshs = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]

optimal_th = []

for nms_thresh in nms_threshs:
    for thresh in threshs:
        
        dfTh = results[(results["nms_thresh"] == nms_thresh) & (results["thresh"] == thresh)]
        
        
        optimal_th.append([nms_thresh, thresh, dfTh["diff"].mean(), dfTh["diff_num"].mean()])
        
optimal_th = pd.DataFrame(optimal_th, columns=["nms_thresh", "thresh", "diff", "diff_num"])
optimal_th

,nms_thresh,thresh,diff,diff_num
0,40,0.35,0.138467,11920.714286
1,40,0.40,0.129093,11084.142857
2,40,0.45,0.118875,10268.571429
3,40,0.50,0.109118,9443.571429
4,40,0.55,0.099105,8563.857143
5,40,0.60,0.088892,7566.142857
6,40,0.65,0.077824,6341.285714
7,40,0.70,0.065648,4662.714286
8,40,0.75,0.073628,2581.571429
9,40,0.80,0.085860,1095.571429


In [14]:
dfTh = results[(results["nms_thresh"] == 40) & (results["thresh"] == 0.80)]
dfTh["diff_rel"] = (dfTh["diff_num"] / dfTh["num_gt"]) * 100

dfTh.sort_values("diff_rel", ascending=False)

d:\anaconda3\envs\eiph_wsi\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,image_name,nms_thresh,thresh,gt,pred,diff,num_gt,num_pred,diff_num,diff_rel
81,BAL_Cat_Turnbull blue_2.svs,40,0.8,0.917399,0.878533,0.038866,569,2972,2403,422.319859
21,BAL_Cat_Turnbull blue_6.svs,40,0.8,0.229252,0.040744,0.188508,3904,5375,1471,37.679303
69,BAL_Cat_Turnbull blue_10.svs,40,0.8,0.484772,0.564961,0.080189,788,508,280,35.532995
9,BAL_Cat_Turnbull blue_13.svs,40,0.8,0.148676,0.129015,0.019661,5475,3643,1832,33.461187
33,BAL_Cat_Turnbull blue_1.svs,40,0.8,0.665907,0.736795,0.070888,3511,4506,995,28.339504
57,BAL_Cat_Turnbull blue_14.svs,40,0.8,0.172664,0.025852,0.146812,2890,2553,337,11.660900
45,BAL_Cat_Turnbull blue_12.svs,40,0.8,0.422831,0.366735,0.056096,3285,2934,351,10.684932


In [16]:
imageDict = {}

for image_name in annotations["image_name"].unique():
    
    imageDf = annotations[annotations["image_name"] == image_name]
    
    image_id = list(imageDf["image_id"])[0]
    image_set = list(imageDf["image_set"])[0]
     
    imageDict[image_name] = {"image_set": image_set, "image_id": image_id}

In [17]:
nms_thresh, thresh = 40, 0.80

inference = {}

result_annos = []
for file in tqdm(folds):
    
    image_id = imageDict[file]["image_id"]
    image_set = imageDict[file]["image_set"]
    
    boxes = np.array(folds[file])
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]
    
    for anno in annos:
        grade = anno[4]
        vector = {"x1":max(0,anno[0]), "y1": max(0, anno[1]), "x2": anno[2], "y2": anno[3]}
        
        result_annos.append([0, image_id, image_set, file, grade, vector, "/", 70, False])
    
    inference[file] = annos
  
result_annos = pd.DataFrame(result_annos, columns=["id","image_id","image_set","image_name","grade","vector","unique_identifier","user_id","deleted"])
result_annos.head()

result_annos.to_pickle("../../Statistics/EIPH_Cat_SDATA_Inference_Annotations_V2.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.56it/s]
